In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from interop_data_plot import read_interop_data
from interop_data_plot import read_runinfo_xml
from interop_data_plot import calculate_phasing_stats

In [ ]:
seqrun_id = '{{ SEQRUN_IGF_ID }}'
seqrun_training_data_csv = '{{ SEQRUN_TRAINING_DATA }}'
new_seqrun_interop_dump = '{{ NEW_SEQRUN_INTEROP_DUMP }}'
new_seqrun_runinfo_xml = '{{ NEW_SEQRUN_RUNINFO_XML }}'

In [ ]:
merged_df = pd.read_csv(seqrun_training_data_csv)
merged_df['q30_pct_mean'] = merged_df['q30_pct_mean'].astype(float)
merged_df['total_median'] = merged_df['total_median'].astype(int)
merged_df['median_qscore'] = merged_df['median_qscore'].astype(int)
merged_df['cluster_count_pct'] = merged_df['cluster_count_pct'].astype(float)
merged_df['densityPf_pct'] = merged_df['densityPf_pct'].astype(float)
merged_df['MaxIntensity_A'] = merged_df['MaxIntensity_A'].astype(float)
merged_df['MaxIntensity_G'] = merged_df['MaxIntensity_G'].astype(float)
merged_df['MaxIntensity_T'] = merged_df['MaxIntensity_T'].astype(float)
merged_df['MaxIntensity_C'] = merged_df['MaxIntensity_C'].astype(float)
merged_df['Focus_A'] = merged_df['Focus_A'].astype(float)
merged_df['Focus_G'] = merged_df['Focus_G'].astype(float)
merged_df['Focus_T'] = merged_df['Focus_T'].astype(float)
merged_df['Focus_C'] = merged_df['Focus_C'].astype(float)
merged_df['error_rate'] = merged_df['error_rate'].astype(float)
merged_df['phasing_slope'] = merged_df['phasing_slope'].astype(float)
merged_df['phasing_offset'] = merged_df['phasing_offset'].astype(float)
merged_df['prephasing_slope'] = merged_df['prephasing_slope'].astype(float)
merged_df['prephasing_offset'] = merged_df['prephasing_offset'].astype(float)
merged_df['obs_failed'] = merged_df['obs_failed'].astype(int)
targets = merged_df['obs_failed'].values
merged_df.drop(['lane_id','is_sc', 'is_failed','seqrun_id','obs_failed'],axis=1,inplace=True)
X_train, X_test, y_train, y_test = train_test_split(merged_df, targets, random_state=0)

In [ ]:
ct = ColumnTransformer([(
    'scaled',StandardScaler(),
    ['q30_pct_mean', 'total_median', 'median_qscore', 'cluster_count_pct',
     'densityPf_pct', 'MaxIntensity_A', 'MaxIntensity_G', 'MaxIntensity_T',
     'MaxIntensity_C', 'Focus_A', 'Focus_G', 'Focus_T', 'Focus_C',
     'error_rate', 'phasing_slope', 'phasing_offset', 'prephasing_slope',
     'prephasing_offset'])])
ct.fit(X_train)
X_train_scaled = ct.transform(X_train)
X_test_scaled = ct.transform(X_test)

In [ ]:
model = RandomForestClassifier(random_state=42,max_depth=7)
model.fit(X_train_scaled ,y_train)
print('Training score: {0:.3f}'.format(model.score(X_train_scaled,y_train)))
print('Test score: {0:.3f}'.format(model.score(X_test_scaled,y_test)))
precision_rf,recall_rf,threshold_rf = precision_recall_curve(y_test,model.predict_proba(X_test_scaled)[:,1])

In [ ]:
plt.plot(precision_rf,recall_rf,label="precision recall curve")
plt.xlabel("Precision")
plt.ylabel("Recall")

In [ ]:
print('Average precision score: {0:.3f}'.format(average_precision_score(y_test,model.predict_proba(X_test_scaled)[:,1])))
print('F1 score: {0:.3f}'.format(f1_score(y_test,model.predict(X_test_scaled))))
fpr,tpr,thresholds = \
  roc_curve(y_test,model.predict_proba(X_test_scaled)[:,1])

In [ ]:
plt.plot(fpr,tpr,label='ROC')
plt.xlabel("FPR")
plt.ylabel("TPR (recall)")

In [ ]:
print('Area under curve: {0:.3f}'.format(roc_auc_score(y_test,model.predict_proba(X_test_scaled)[:,1])))

In [ ]:
confusion = confusion_matrix(y_test,model.predict(X_test_scaled))
confusion

In [ ]:
def get_run_matircs(seqrun_id,interop_dump,runinfo_xml):
  try:
    (tile,q2030,extraction,error,empiricalPhasing,correctedInt,qByLane) = \
      read_interop_data(filepath=interop_dump)
    runinfoDf = read_runinfo_xml(runinfo_xml)
    cycles_list = \
      runinfoDf[runinfoDf['index_read']=='N'][
        ['read_id','cycles','start_cycle']].\
        to_dict(orient='records')
    non_index_cycles = list()
    non_index_ids = list()
    for entry in cycles_list:
      read_id = entry.get('read_id')
      cycles = entry.get('cycles')
      start_cycle = entry.get('start_cycle')
      finish_cycle = start_cycle+cycles
      non_index_cycles.extend([
          i for i in range(start_cycle+1,finish_cycle)])
      non_index_ids.append(int(read_id))
    q2030['Lane'] = q2030['Lane'].astype(int)
    q2030['Cycle'] = q2030['Cycle'].astype(int)
    q2030['Tile'] = q2030['Tile'].astype(int)
    q2030['Q30'] = q2030['Q30'].astype(int)
    q2030['Total'] = q2030['Total'].astype(int)
    q2030['MedianQScore'] = q2030['MedianQScore'].astype(int)
    q2030df = list()
    for lane_id,l_data in q2030.groupby('Lane'):
      q30_median = \
        l_data[l_data['Cycle'].isin(non_index_cycles)].\
        groupby('Tile')['Q30'].\
        agg('median').mean()
      total_median = \
        l_data[l_data['Cycle'].isin(non_index_cycles)].\
        groupby('Tile')['Total'].\
        agg('median').mean()
      q30_pct = q30_median / total_median
      median_qscore_mean = \
        l_data[l_data['Cycle'].isin(non_index_cycles)].\
        groupby('Tile')['MedianQScore'].\
        agg('median').mean()
      row = {
        'lane_id':lane_id,
        'q30_pct_mean':q30_pct,
        'total_median':total_median,
        'median_qscore':median_qscore_mean}
      q2030df.append(row)
    q2030df = pd.DataFrame(q2030df).fillna(0)
    tile_filt = tile[tile['Lane'].isin(['1','2','3','4','5','6','7','8'])].copy()
    tile_filt['Lane'] = tile_filt['Lane'].astype(int)
    tile_filt['Read'] = tile_filt['Read'].astype(int)
    tile_filt['ClusterCount'] = tile_filt['ClusterCount'].astype(float)
    tile_filt['ClusterCountPF'] = tile_filt['ClusterCountPF'].astype(float)
    tile_filt['Density'] = tile_filt['Density'].astype(float)
    tile_filt['DensityPF'] = tile_filt['DensityPF'].astype(float)
    tiledf = list()
    for lane_id,l_data in tile_filt.groupby('Lane'):
      l_data_sum = \
        l_data[l_data['Read'].isin(non_index_ids)].groupby('Tile')[
          ['ClusterCount','ClusterCountPF','Density','DensityPF']].\
          agg('sum').mean()
      cluster_countPf_pct = \
        l_data_sum['ClusterCountPF'] / l_data_sum['ClusterCount']
      densityPf_pct = \
        l_data_sum['DensityPF'] / l_data_sum['Density']
      row = {
        'lane_id':lane_id,
        'cluster_count_pct':cluster_countPf_pct,
        'densityPf_pct':densityPf_pct}
      tiledf.append(row)
    tiledf = pd.DataFrame(tiledf).fillna(0)
    extraction['Lane'] = extraction['Lane'].astype(int)
    extraction['Tile'] = extraction['Tile'].astype(int)
    extraction['Cycle'] = extraction['Cycle'].astype(int)
    extraction['MaxIntensity_A'] = extraction['MaxIntensity_A'].astype(int)
    extraction['MaxIntensity_T'] = extraction['MaxIntensity_T'].astype(int)
    extraction['MaxIntensity_G'] = extraction['MaxIntensity_G'].astype(int)
    extraction['MaxIntensity_C'] = extraction['MaxIntensity_C'].astype(int)
    extraction['Focus_A'] = extraction['Focus_A'].astype(float)
    extraction['Focus_T'] = extraction['Focus_T'].astype(float)
    extraction['Focus_G'] = extraction['Focus_G'].astype(float)
    extraction['Focus_C'] = extraction['Focus_C'].astype(float)
    extractiondf = list()
    extraction_columns = [
      'MaxIntensity_A','MaxIntensity_G','MaxIntensity_T','MaxIntensity_C',
      'Focus_A','Focus_G','Focus_T','Focus_C']
    for lane_id,l_data in extraction.groupby('Lane'):
      e_data_mean = \
        l_data[l_data['Cycle'].isin(non_index_cycles)].\
        groupby('Tile')[extraction_columns].\
        agg('median').mean()
      row = {'lane_id':lane_id}
      for c in extraction_columns:
        row.update({c:e_data_mean.get(c)})
      extractiondf.append(row)
    extractiondf = pd.DataFrame(extractiondf).fillna(0)
    error['Lane'] = error['Lane'].astype(int)
    error['Tile'] = error['Tile'].astype(int)
    error['Cycle'] = error['Cycle'].astype(int)
    error['ErrorRate'] = error['ErrorRate'].astype(float)
    errordf = list()
    for lane_id,l_data in error.groupby('Lane'):
      l_data_mean = \
        l_data[l_data['Cycle'].isin(non_index_cycles)].\
        groupby('Tile')[['ErrorRate']].\
        agg('median').mean()
      error_rate = l_data_mean['ErrorRate']
      errordf.append({'lane_id':lane_id,'error_rate':error_rate})
    if len(errordf)>0:
      errordf = pd.DataFrame(errordf).fillna(0)
    else:
      errordf = pd.DataFrame(columns=['lane_id','error_rate'])
    phasing_data = \
      calculate_phasing_stats(empiricalPhasing,runinfoDf)
    phasing_columns = [
      'phasing_slope','phasing_offset',
      'prephasing_slope','prephasing_offset']
    for c in phasing_columns:
      phasing_data[c] = phasing_data[c].astype(float)
    phasing_df = list()
    for lane_id,l_data in phasing_data.groupby('lane_id'):
      p_data = \
        l_data[l_data['read_id'].isin([1,4])][phasing_columns].mean()
      row = {'lane_id':lane_id}
      for c in phasing_columns:
        row.update({c:p_data.get(c)})
      phasing_df.append(row)
    phasing_df = pd.DataFrame(phasing_df).fillna(0)
    merged_df = \
      q2030df.merge(tiledf,on='lane_id',how='left').\
      merge(extractiondf,on='lane_id',how='left').\
      merge(errordf,on='lane_id',how='left').\
      merge(phasing_df,on='lane_id',how='left').\
      fillna(0)
    merged_df['seqrun_id'] = seqrun_id
    return merged_df
  except:
    raise

In [ ]:
df = \
  get_run_matircs(
    seqrun_id=seqrun_id,
    interop_dump=new_seqrun_interop_dump,
    runinfo_xml=new_seqrun_runinfo_xml)
seqrun_ids = df[['seqrun_id','lane_id']].copy()
df.drop(['lane_id','seqrun_id'],axis=1,inplace=True)
df_scaled = ct.transform(df)
seqrun_ids['is_low'] = model.predict(df_scaled)

In [ ]:
def color_seqrun_prediction(s):
    if s.is_low == 1:
        bgcolors = ['','','background:crimson']
    else:
        bgcolors = ['','','background:lightgreen']
    return bgcolors

In [ ]:
seqrun_ids.style.apply(lambda s: color_seqrun_prediction(s),axis=1)